Lets perform the transfer learning


# Download the dataset

In [40]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2020-07-05 09:00:14--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.203.128, 74.125.204.128, 64.233.189.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M   124MB/s    in 0.5s    

2020-07-05 09:00:15 (124 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



# Dataset preprocessing
## import the dependencies

In [41]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.2.0'

## unzip the downloaded data

In [42]:
dataset_path = "./cats_and_dogs_filtered.zip"
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")
zip_object.extractall("./")
zip_object.close()

In [43]:
dataset_path_new = "./cats_and_dogs_filtered/"

In [44]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

# Building the model

# Loading the pretrained model (MobileNetV2)

In [45]:
IMG_SHAPE = (128, 128, 3)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")
# base_model.summary()

## Freezing the base model

In [46]:
base_model.trainable = False

## Defining the custome head i.e. the last layer of out network

In [47]:
base_model.output

<tf.Tensor 'out_relu_2/Identity:0' shape=(None, 4, 4, 1280) dtype=float32>

In [48]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [49]:
global_average_layer

<tf.Tensor 'global_average_pooling2d_2/Identity:0' shape=(None, 1280) dtype=float32>

In [50]:
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

## Defining the model

In [51]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)
#model.summary

## Compiling the model

In [52]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

## Creating Data Generators
Resizing images

In [53]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [54]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


## Training the model

In [55]:
model.fit(train_generator, epochs=5, validation_data=valid_generator)

Epoch 1/5
16/16 [==============================] - 36s 2s/step - loss: 0.7056 - accuracy: 0.6120 - val_loss: 0.6399 - val_accuracy: 0.6460
Epoch 2/5
16/16 [==============================] - 35s 2s/step - loss: 0.5804 - accuracy: 0.7065 - val_loss: 0.5449 - val_accuracy: 0.7340
Epoch 3/5
16/16 [==============================] - 36s 2s/step - loss: 0.4990 - accuracy: 0.7725 - val_loss: 0.4739 - val_accuracy: 0.7870
Epoch 4/5
16/16 [==============================] - 35s 2s/step - loss: 0.4374 - accuracy: 0.8250 - val_loss: 0.4155 - val_accuracy: 0.8370
Epoch 5/5
16/16 [==============================] - 35s 2s/step - loss: 0.3866 - accuracy: 0.8510 - val_loss: 0.3672 - val_accuracy: 0.8750


## Transfer learning model evaluation

In [56]:
valid_loss, valid_accuracy = model.evaluate(valid_generator)
print("Accuracy after transfer learning: {}".format(valid_accuracy))

8/8 [==============================] - 10s 1s/step - loss: 0.3672 - accuracy: 0.8750
Accuracy after transfer learning: 0.875


# Fine tuning

## Un-freeze a few top layers from the model

In [57]:
base_model.trainable = True

In [58]:
print("Number of layersin the base model: {}".format(len(base_model.layers)))

Number of layersin the base model: 155


In [59]:
fine_tune_at = 100

In [60]:
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

## Compiling the model for fine-*tuning*

In [61]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

## fine tuning

In [62]:
model.fit(train_generator,  
                    epochs=5, 
                    validation_data=valid_generator)

Epoch 1/5
16/16 [==============================] - 58s 4s/step - loss: 0.2126 - accuracy: 0.9065 - val_loss: 0.1408 - val_accuracy: 0.9490
Epoch 2/5
16/16 [==============================] - 57s 4s/step - loss: 0.0308 - accuracy: 0.9970 - val_loss: 0.1326 - val_accuracy: 0.9550
Epoch 3/5
16/16 [==============================] - 57s 4s/step - loss: 0.0116 - accuracy: 1.0000 - val_loss: 0.1697 - val_accuracy: 0.9510
Epoch 4/5
16/16 [==============================] - 57s 4s/step - loss: 0.0033 - accuracy: 1.0000 - val_loss: 0.1652 - val_accuracy: 0.9520
Epoch 5/5
16/16 [==============================] - 57s 4s/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.2511 - val_accuracy: 0.9480


## Evaluating the fine tuned model

In [63]:
valid_loss, valid_accuracy = model.evaluate(valid_generator)
print("Validation accuracy after fine tuning: {}".format(valid_accuracy))

8/8 [==============================] - 10s 1s/step - loss: 0.2511 - accuracy: 0.9480
Validation accuracy after fine tuning: 0.9480000138282776
